In [1]:
import pandas as pd


In [2]:
df= pd.read_csv('airlines_reviews.csv')
df.head(4)

,Title,Name,Review Date,Airline,Verified,Reviews,Type of Traveller,Month Flown,Route,Class,Seat Comfort,Staff Service,Food & Beverages,Inflight Entertainment,Value For Money,Overall Rating,Recommended
0,Flight was amazing,Alison Soetantyo,2024-03-01,Singapore Airlines,True,Flight was amazing. The crew onboard this fl...,Solo Leisure,December 2023,Jakarta to Singapore,Business Class,4,4,4,4,4,9,yes
1,seats on this aircraft are dreadful,Robert Watson,2024-02-21,Singapore Airlines,True,Booking an emergency exit seat still meant h...,Solo Leisure,February 2024,Phuket to Singapore,Economy Class,5,3,4,4,1,3,no
2,Food was plentiful and tasty,S Han,2024-02-20,Singapore Airlines,True,Excellent performance on all fronts. I would...,Family Leisure,February 2024,Siem Reap to Singapore,Economy Class,1,5,2,1,5,10,yes
3,“how much food was available,D Laynes,2024-02-19,Singapore Airlines,True,Pretty comfortable flight considering I was f...,Solo Leisure,February 2024,Singapore to London Heathrow,Economy Class,5,5,5,5,5,10,yes


In [3]:
x=df['Reviews']
y=df['Recommended']

In [4]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [5]:
print('shape of x_train',x_train.shape)
print('shape of x_test',x_test.shape)
print('shape of y_train',y_train.shape)
print('shape of y_test',y_test.shape)

shape of x_train (6480,)
shape of x_test (1620,)
shape of y_train (6480,)
shape of y_test (1620,)


In [6]:
print(x_train[6])
print(y_train[6])

  Boarding process went smoothly, and plane left on time. I had a window seat in the first part of economy section. The plane has 3-4-3 layout. We were 3 adults on this row, and it was very narrow. When the passenger in front of me, reclined his seat, i could hardly look at my screen, and could not fold out the table. The crew told people to raise up the seat backs during the meal times. We had two meals, dinner and breakfast, both meals were ok. I wanted to charge my phone during the flight, usb charger wad not working. I remember many years ago, when travelling with Singapore Airlines, the drink trolley came before the meal, and again when you got the meal, and they showed up again shortly after the meal also. Now you just get drinks along with the meal, and they do not ask if you want a refill. Very ordinary service and not very friendly staff either.
no


In [7]:
!pip install --upgrade tensorflow


In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer


In [9]:
!pip install keras


In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=1000, lower=True)  
tokenizer.fit_on_texts(x_train) 
x_train = tokenizer.texts_to_sequences(x_train) 
x_test = tokenizer.texts_to_sequences(x_test)

vocab_size = len(tokenizer.word_index) + 1  

print("Vocabulary size:", vocab_size)


Vocabulary size: 19077


In [11]:
print(x_train[6])
print(y_train[6])

[2, 183, 492, 12, 548, 80, 107, 100, 217, 467, 394, 5, 25, 98, 268, 159, 294, 34, 4, 39, 12, 6, 247, 297, 548, 3, 302, 53, 45, 306, 451, 3, 457, 1, 9, 4, 49, 8, 1, 152, 120, 357, 67, 435, 99, 10]
no


In [12]:
from keras.preprocessing.sequence import pad_sequences 

maxlen = 100 
x_train = pad_sequences(x_train, padding='post', maxlen=maxlen) 
x_test = pad_sequences(x_test, padding='post', maxlen=maxlen)  
print(x_train[3, :])


[  1  73  32  17   5 145   6 757  10  54 415  21   5  25   2   6 757  11
 392   5   4   8 903   8   1 129  24   2 333 392 167  58 130 688   3 656
  10   1 173 581   3 108  10 165   1 124   1  26   4 130 321 462 451   3
  29 249  12  40 692  19 180  66   1  36 293   4 314  21  16 794   1  34
 134   4  39   5 118 405 286 438  12 392  41  44   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [13]:
from keras.utils import to_categorical
label_mapping = {'no': 0, 'yes': 1}  
y_train = y_train.map(label_mapping)  
y_test = y_test.map(label_mapping)

# Number of classes
num_classes = len(label_mapping)

# Apply one-hot encoding
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Verify shapes and content
print(y_train.shape)
print(y_train[0])


(6480, 2)
[0. 1.]


In [14]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, SimpleRNN
from keras import optimizers
from keras.preprocessing.sequence import pad_sequences
import numpy as np

x_train = np.array(x_train).reshape((x_train.shape[0], x_train.shape[1], 1))
print(x_train.shape)  # Example shape: (750, 100, 1)

x_test = np.array(x_test).reshape((x_test.shape[0], x_test.shape[1], 1))
print(x_test.shape)  # Example shape: (250, 100, 1)


(6480, 100, 1)
(1620, 100, 1)


In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.layers import Input  # Import Input layer

def create_model(vocab_size, embedding_dim, maxlen):
  # Create an input layer with the specified shape
  inputs = Input(shape=(maxlen,))

  # Add the embedding layer
  embedding_layer = Embedding(input_dim=vocab_size, 
                             output_dim=embedding_dim)(inputs)

  lstm_layer = LSTM(64, dropout=0.2, recurrent_dropout=0.2)(embedding_layer)
  dense_layer = Dense(2, activation='softmax')(lstm_layer)

  # Create the model
  model = tf.keras.Model(inputs=inputs, outputs=dense_layer)

  # Print model summary
  model.summary()

  # Compile the model
  model.compile(loss='categorical_crossentropy', 
                optimizer='adam', 
                metrics=['accuracy'])

  return model
vocab_size = len(tokenizer.word_index) + 1 
embedding_dim = 50
maxlen = 100

# Create the model
model = create_model(vocab_size, embedding_dim, maxlen) 

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 50)        │       953,850 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        29,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 983,420 (3.75 MB)

 Trainable params: 983,420 (3.75 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))


Epoch 1/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 7s 56ms/step - accuracy: 0.5723 - loss: 0.6719 - val_accuracy: 0.7623 - val_loss: 0.5452
Epoch 2/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 58ms/step - accuracy: 0.8041 - loss: 0.4832 - val_accuracy: 0.8568 - val_loss: 0.3952
Epoch 3/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 57ms/step - accuracy: 0.8196 - loss: 0.4471 - val_accuracy: 0.7395 - val_loss: 0.5610
Epoch 4/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 58ms/step - accuracy: 0.7520 - loss: 0.5374 - val_accuracy: 0.7253 - val_loss: 0.5690
Epoch 5/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 58ms/step - accuracy: 0.7344 - loss: 0.5445 - val_accuracy: 0.8235 - val_loss: 0.4433
Epoch 6/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 56ms/step - accuracy: 0.7544 - loss: 0.5462 - val_accuracy: 0.7259 - val_loss: 0.5365
Epoch 7/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 57ms/step - accuracy: 0.7423 - loss: 0.5138 - val_accuracy: 0.7469 - val_loss: 0.4896
Epoch 8/10
102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 58ms/step - accuracy: 0.7637 - loss: 0.4827 - val_accu

In [17]:
# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8105 - loss: 0.4693
Test Loss: 0.43784183263778687
Test Accuracy: 0.8358024954795837


In [18]:
a = ["The food was absolutely  not delicious  !"]
# Tokenize and pad the example input
a = tokenizer.texts_to_sequences(a)
a = pad_sequences(a, padding='post', maxlen=maxlen)

# Predict the sentiment for the example input
prediction = model.predict(a)

# Print the prediction (0 means negative, 1 means positive)
predicted_class = np.argmax(prediction, axis=1)
print(f"Predicted sentiment class: {predicted_class[0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
Predicted sentiment class: 0
